# Numerical Computing with NumPy

## Introductory example

Python built-in collections like `list` offer a flexible way of storing and maniupulating data. As dicussed previously, collections usually just store references to objects. While this is every convenient when writing code, it comes with costs in performance in memory. 

Let's look at an example. Say we took one million measurements in an experiment and now want to compute the mean of it. We could do it in the following manner. 

In [1]:
import random 
measurements = [random.randint(150, 200) for _ in range(1_000_000)]

def calculate_mean(measurements):
    accumulator = 0
    for measurement in measurements:
        accumulator += measurement
    
    mean = accumulator / len(measurements)
    return mean

%timeit calculate_mean(measurements)

147 ms ± 7.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


This is rather slow since Python has to rebind a new variable in every loop and then has to check whether the `+` operation is supported between the `accumulator` and the current `measurement`. This prevents it from trying to add together objects that can't be added, but in this case we are pretty sure that we are only dealing with integers. If we could tell the interpreter that we are only adding integers, we could skip all that typechecking and speed up the operation. For this purpose, `numpy` was invented.  


To use numpy we have to import it. The import is usuall aliased as `np` so we have to type less later on. Aliasing things is only recommended if it is well established in the community of the respective package.

In [2]:
import numpy as np

Numpy's standard datatype is the `ndarray` (which stands for n-dimensional array). In the simplest case, numpy array can be created from list.

In [3]:
measurements_array = np.array(measurements)
measurements_array

array([150, 163, 161, ..., 153, 172, 150])

In [4]:
type(measurements_array)

numpy.ndarray

They behave very similar to list, but have a fixed datatype underneath. Numpy automatically notices that all our values are intergers and chooses the appropriate datatype. An integer that takes up 64 bits of memory.

In [5]:
measurements_array[0]

150

In [6]:
measurements_array[0:5]

array([150, 163, 161, 191, 175])

In [7]:
measurements_array.dtype

dtype('int64')

Moreover, numpy offers a lot of routines for mathematical operations of arrays. Let's see if we acually gained something by using numpy.

In [8]:
%timeit np.mean(measurements_array)

4.48 ms ± 94.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Almost 100x speedup in comparision to the pure Python implementation! After convincing ourselfs that NumPy is useful, we have a more in depth look at the numpy array.

## Anatomy of arrays
Every array has a bunch of attributes that yield inforation about what it is.

### dtype

`.dtype` gives information about the data type. arrays can contain bools, ints, unsigned ints, floats or complex numbers of various byte sizes. They can also store strings or Python objects, but that has very few use cases.

In [9]:
values = [0, 1, 2, 3, 4]
int_arr = np.array(values, dtype='int')
int_arr, int_arr.dtype

(array([0, 1, 2, 3, 4]), dtype('int64'))

If the dtype does not match the given values, numpy will cast everything to that data type.

In [10]:
bool_arr = np.array(values, dtype='bool')
bool_arr, bool_arr.dtype

(array([False,  True,  True,  True,  True]), dtype('bool'))

If no explicit data type is given, numpy will choose the "smallest common denominator". In the following example, everything becomes a float, as ints can be represented as floats, but not vice versa.

In [11]:
values = [0, 1, 2.5, 3, 4]
float_arr = np.array(values)
float_arr, float_arr.dtype

(array([0. , 1. , 2.5, 3. , 4. ]), dtype('float64'))

However, once the data type is set, everything will be coerced to that type.

In [12]:
int_arr[1] = 2.5
int_arr, int_arr.dtype

(array([0, 2, 2, 3, 4]), dtype('int64'))

These non-Python data types force us to again think about problems like overflow etc.

In [13]:
values = [0, 1, 2, 3, 4]
uint_arr = np.array(values, dtype='int8')
uint_arr, uint_arr.dtype

(array([0, 1, 2, 3, 4], dtype=int8), dtype('int8'))

In [14]:
uint_arr[1] += 255
uint_arr

array([0, 0, 2, 3, 4], dtype=int8)

### shape and ndim
`.shape` is very important for keeping track of arrays with more than one dimension. It is a tuple with the number of elementns in each dimension. `.ndim` is just the number of dimensions in total. 

In [15]:
values = [0, 1, 2, 3, 4]
one_dim_arr = np.array(values)
one_dim_arr

array([0, 1, 2, 3, 4])

In [16]:
one_dim_arr.shape

(5,)

In [17]:
one_dim_arr.ndim

1

In [18]:
values = [[0, 1, 2, 3, 4]] * 3
two_dim_arr = np.array(values)
two_dim_arr

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [19]:
two_dim_arr.shape

(3, 5)

In [20]:
two_dim_arr.ndim

2

In [21]:
values = [[[0, 1, 2, 3, 4]] * 3] * 6
three_dim_arr = np.array(values)
three_dim_arr

array([[[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]]])

In [22]:
three_dim_arr.shape

(6, 3, 5)

In [23]:
three_dim_arr.ndim

3

## Creating arrays
We already saw how arrays can be created from Python lists (the same works with tuples). However, we often would like to create arrays directly, without creating Python objects. This can be accomplished by several utility functions.

The equivalent of `range`.

In [24]:
np.arange(9)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [25]:
np.arange(start=2, stop=13, step=2)

array([ 2,  4,  6,  8, 10, 12])

Creating an array with a certain number of values in a certain interval.

In [26]:
np.linspace(start=-5, stop=5, num=10)

array([-5.        , -3.88888889, -2.77777778, -1.66666667, -0.55555556,
        0.55555556,  1.66666667,  2.77777778,  3.88888889,  5.        ])

An array containing zeros. The default `dtype` is `float`.

In [27]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

`np.zeros` takes a `shape` argument that lets us create multidimensional arrays.

In [28]:
np.zeros(shape=(2, 3, 2))

array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])

The same goes for `ones`, `empty` and `full`.

In [29]:
np.ones(shape=(2, 3, 2))

array([[[1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.]]])

In [30]:
# Corresponds to whatever was left in memory. Using zeros for initialising arrays is usually saver.
np.empty(shape=(2, 3, 2))

array([[[1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.]]])

In [31]:
np.full(shape=(2, 3, 2), fill_value=42)

array([[[42, 42],
        [42, 42],
        [42, 42]],

       [[42, 42],
        [42, 42],
        [42, 42]]])

`np.random` contains a lot of functions to create arrays filled with random values of various probability distributions.

In [32]:
np.random.random((3, 3))

array([[0.33968612, 0.67013351, 0.05401438],
       [0.23501536, 0.8596346 , 0.68410634],
       [0.27947453, 0.13671571, 0.32125113]])

## Mathematical operations
Numpy contains a lot of mathematical functions that operate on arrays in a vectorized manner. That means that they are applied to each element, without explicit for-loops. Vectorized functions are called `ufuncs` (universal functions) in Numpy.

### Standard arithmetic

In [33]:
arr = np.arange(9)
arr

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [34]:
arr + arr

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16])

In [35]:
arr - arr

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:
arr / arr

/home/jarno/.miniconda/envs/scientific/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([nan,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [37]:
arr * arr

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64])

In [38]:
arr ** 2

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64])

Using `@` you can even do matrix multiplication. In the case of 1d arrays, this is the inner product between two vectors.

In [39]:
arr @ arr

204

In [40]:
# That's the same as
np.sum(arr * arr)

204

### Some standard functions

In [41]:
np.log(arr)

/home/jarno/.miniconda/envs/scientific/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


array([      -inf, 0.        , 0.69314718, 1.09861229, 1.38629436,
       1.60943791, 1.79175947, 1.94591015, 2.07944154])

In [42]:
np.exp(arr)

array([1.00000000e+00, 2.71828183e+00, 7.38905610e+00, 2.00855369e+01,
       5.45981500e+01, 1.48413159e+02, 4.03428793e+02, 1.09663316e+03,
       2.98095799e+03])

In [43]:
np.sin(arr)

array([ 0.        ,  0.84147098,  0.90929743,  0.14112001, -0.7568025 ,
       -0.95892427, -0.2794155 ,  0.6569866 ,  0.98935825])

Always try to use vectorized ufuncs instead of explicit loops!

### Aggregations functions
Aggregation function are functions that reduce the dimensionality of an array. They provide an `axis` argument, to specify which dimension to reduce.

In [44]:
two_dim_arr = np.arange(16).reshape((4, 4))
two_dim_arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [45]:
np.min(two_dim_arr)

0

In [46]:
np.min(two_dim_arr, axis=0)

array([0, 1, 2, 3])

In [47]:
np.min(two_dim_arr, axis=1)

array([ 0,  4,  8, 12])

In [48]:
np.max(two_dim_arr)

15

In [49]:
np.max(two_dim_arr, axis=0)

array([12, 13, 14, 15])

In [50]:
np.max(two_dim_arr, axis=1)

array([ 3,  7, 11, 15])

In [51]:
np.sum(two_dim_arr)

120

In [52]:
np.sum(two_dim_arr, axis=0)

array([24, 28, 32, 36])

In [53]:
np.sum(two_dim_arr, axis=1)

array([ 6, 22, 38, 54])

Many of these function are also available as method on the array object.

In [54]:
two_dim_arr.sum()

120

### Broadcasting
What happens if you try to add arrays of different shapes? Numpy will try to expand the arrays according to three rules and try to make their shapes match, so the operation can be applied elementwise. 

**1. Rule** If the arrays have different numbers of dimensions, the smaller shape is padded with ones on its left side.<br/>
            Example: (5 x 3) + (3) &rarr; (5 x 3) + (**1** x 3)<br/>
**2. Rule** If the number of the dimensions matches, but the size of a dimension does not, dimensions with the size of 1 are expanded.<br/>
            Example: (5 x 3) + (1 x 3) &rarr; (5 x 3) + (**5** x 3)<br/>
**3. Rule** If the shapes of the  arrays still defer after applying the Rule 1 and 2, a broadcasting error is raised.

The figure below gives an illustration (source https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html) ![](broadcasting.png)




The Numpy documentation gives further insights https://docs.scipy.org/doc/numpy-1.14.0/user/basics.broadcasting.html. 


In [55]:
a = np.arange(15).reshape((5, 3))
a

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [56]:
b = np.arange(3)
b

array([0, 1, 2])

In [57]:
a + b 

array([[ 0,  2,  4],
       [ 3,  5,  7],
       [ 6,  8, 10],
       [ 9, 11, 13],
       [12, 14, 16]])

Here is a case in which broadcasting fails.

In [58]:
c = np.arange(4)
a + c

ValueError: operands could not be broadcast together with shapes (5,3) (4,) 

## Advanced indexing
Numpy provides indexing methods that go beyond the indexing techniques known from standard Python sequences.


### Multidimensional indexing
Instead of doing subsequent indexing as with standard Python lists you can index all dimensions at once.

In [59]:
two_dim_list = [
    [ 0,  1,  2],
    [ 3,  4,  5],
    [ 6,  7,  8],
    [ 9, 10, 11],
    [12, 13, 14]
]
two_dim_list[2][1]

7

In [60]:
inner_list = two_dim_list[2]
inner_list[1]

7

In [61]:
two_dim_arr = np.array(two_dim_list)
two_dim_arr[2, 1]

7

In [62]:
two_dim_arr = np.array(two_dim_list)
two_dim_arr[2, 1]

7

You can use a colon to get all values from that dimensions.

In [63]:
large_two_dim_arr = np.arange(81).reshape((9, 9))
large_two_dim_arr

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23, 24, 25, 26],
       [27, 28, 29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49, 50, 51, 52, 53],
       [54, 55, 56, 57, 58, 59, 60, 61, 62],
       [63, 64, 65, 66, 67, 68, 69, 70, 71],
       [72, 73, 74, 75, 76, 77, 78, 79, 80]])

In [64]:
large_two_dim_arr[:, 1]

array([ 1, 10, 19, 28, 37, 46, 55, 64, 73])

Standard slicing with `(start, stop, step)` works as expected.

In [65]:
large_two_dim_arr[:, 1:3]

array([[ 1,  2],
       [10, 11],
       [19, 20],
       [28, 29],
       [37, 38],
       [46, 47],
       [55, 56],
       [64, 65],
       [73, 74]])

In [66]:
large_two_dim_arr[:, 2:7:2]

array([[ 2,  4,  6],
       [11, 13, 15],
       [20, 22, 24],
       [29, 31, 33],
       [38, 40, 42],
       [47, 49, 51],
       [56, 58, 60],
       [65, 67, 69],
       [74, 76, 78]])

Slices of an array are always `views`. That means, you just "view" the same chunk of meomory from a different perspective. This saves a lot of memory, but it means also that the original array will be changed, if you change the view.

In [67]:
arr_slice = large_two_dim_arr[:, 1]
arr_slice[:] = 0
large_two_dim_arr

array([[ 0,  0,  2,  3,  4,  5,  6,  7,  8],
       [ 9,  0, 11, 12, 13, 14, 15, 16, 17],
       [18,  0, 20, 21, 22, 23, 24, 25, 26],
       [27,  0, 29, 30, 31, 32, 33, 34, 35],
       [36,  0, 38, 39, 40, 41, 42, 43, 44],
       [45,  0, 47, 48, 49, 50, 51, 52, 53],
       [54,  0, 56, 57, 58, 59, 60, 61, 62],
       [63,  0, 65, 66, 67, 68, 69, 70, 71],
       [72,  0, 74, 75, 76, 77, 78, 79, 80]])

If you need all values from several consecutive dimensions you can use ellipsis (`...`) as a shorthand.

In [68]:
# np.stack joins arrays along a new axis.
four_dim_arr = np.stack((np.ones((3, 3, 3)), 
                         np.ones((3, 3, 3)) * 2, 
                         np.ones((3, 3, 3)) * 3, 
                         np.ones((3, 3, 3)) * 4))
four_dim_arr

array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[2., 2., 2.],
         [2., 2., 2.],
         [2., 2., 2.]],

        [[2., 2., 2.],
         [2., 2., 2.],
         [2., 2., 2.]],

        [[2., 2., 2.],
         [2., 2., 2.],
         [2., 2., 2.]]],


       [[[3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.]],

        [[3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.]],

        [[3., 3., 3.],
         [3., 3., 3.],
         [3., 3., 3.]]],


       [[[4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.]],

        [[4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.]],

        [[4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.]]]])

In [69]:
four_dim_arr.shape

(4, 3, 3, 3)

In [70]:
four_dim_arr[3, :, :, :]

array([[[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]],

       [[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]],

       [[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]]])

In [71]:
four_dim_arr[3, ...]

array([[[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]],

       [[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]],

       [[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]]])

### Fancy indexing
You can pass an array containing indices, this especially useful for drawing random items from an array.

In [72]:
arr = np.arange(9) + 10
arr

array([10, 11, 12, 13, 14, 15, 16, 17, 18])

In [73]:
indices = np.array([1, 4, 5])
arr[indices]

array([11, 14, 15])

The resulting array will reflect the shape of the index array.

In [74]:
indices = np.array([[1, 4],
                    [5, 7]])
arr[indices]

array([[11, 14],
       [15, 17]])

You can index each dimension separately.

In [75]:
two_dim_arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [76]:
x_indices = np.array([3, 4])
y_indices = np.array([1, 2])
two_dim_arr[x_indices, y_indices]

array([10, 14])

### Masking
Logical arrays, i.e. arrays containing boolean values, can be used to index other arrays. These logical arrays are then called masks. This is especially useful to index based on logical conditions.

In [77]:
# A simple integer array.
arr = np.arange(1, 6)
arr

array([1, 2, 3, 4, 5])

In [78]:
# A boolean array of the same shape as arr.
mask = np.array([True, False, True, False, True])
mask

array([ True, False,  True, False,  True])

Using the mask for indexing returns an array with only elements at positions where `mask` is `True`.  

In [79]:
arr[mask]

array([1, 3, 5])

The same can be used for assignment, which keeps the shape of the original array.

In [80]:
arr[mask] = 10
arr

array([10,  2, 10,  4, 10])

Mask can be created by using logical operators. For example, to get all the entries in an array that are greater than two.


In [81]:
arr = np.arange(1, 6)
greater_two = arr > 2
greater_two


array([False, False,  True,  True,  True])

In [82]:
arr[greater_two]

array([3, 4, 5])

Or even shorter.

In [83]:
arr[arr > 2]

array([3, 4, 5])

Different masks can be combined using bitwise logical operators. These are the vectorized version of locial operators and should not be confounded with `and`, `or` and `not` with try to evaluated the truth value of a whole object.

In [84]:
smaller_or_equal_four = arr <= 4
smaller_or_equal_four   

array([ True,  True,  True,  True, False])

Bitwise and `&`.

In [85]:
greater_two & smaller_or_equal_four

array([False, False,  True,  True, False])

In [86]:
# This does not work.
greater_two and smaller_or_equal_four

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [87]:
arr[greater_two & smaller_or_equal_four]

array([3, 4])

Bitwise or using `|`.

In [88]:
arr[greater_two | smaller_or_equal_four]

array([1, 2, 3, 4, 5])

Bitwise xor using `^`.

In [89]:
arr[greater_two ^ smaller_or_equal_four]

array([1, 2, 5])

Bitwise negation using `~`.

In [90]:
# Gives everything smaller or equal to 2.
arr[~greater_two]

array([1, 2])

## Extending arrays

### Adding new dimensions with `np.newaxis`
Instead of `np.newaxis`, `None` can be used.

In [91]:
one_dim_arr = np.arange(5)
one_dim_arr, one_dim_arr.shape

(array([0, 1, 2, 3, 4]), (5,))

In [92]:
two_dim_arr = one_dim_arr[np.newaxis, :]
two_dim_arr, two_dim_arr.shape

(array([[0, 1, 2, 3, 4]]), (1, 5))

In [93]:
two_dim_arr = one_dim_arr[:, np.newaxis]
two_dim_arr, two_dim_arr.shape

(array([[0],
        [1],
        [2],
        [3],
        [4]]), (5, 1))

### Combining arrays
There are many ways to combine existing arrays, like `np.append`, `np.concatenate` and `np.stack`. However, these operations always require the whole array to be copied. Therefore, it often makes more sense to allocate an array of the size you need later upfront and then just fill the respective parts.

In [94]:
np.concatenate((one_dim_arr, one_dim_arr))

array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4])

In [95]:
np.stack((one_dim_arr, one_dim_arr))

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [98]:
np.append(one_dim_arr, one_dim_arr)

array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4])

## Further Readings

NumPy chapter from Jake VanderPlas's "Python Data Science Handbook" https://jakevdp.github.io/PythonDataScienceHandbook/02.00-introduction-to-numpy.html

Video tutorial from Scipy 2017


In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('lKcwuPnSHIQ')

In [2]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/lKcwuPnSHIQ" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>')